In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
image_dims = 128
batch_size = 64

In [3]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(image_dims, image_dims, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 64)        0

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
input_path = './chest_xray/'

In [7]:
# Training data generator
training_data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
training_gen = training_data_generator.flow_from_directory(
    directory=input_path+'train',
    target_size=(image_dims, image_dims),
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [8]:
# Validation data generator
validation_data_generator = ImageDataGenerator(rescale=1./255)
validation_gen = validation_data_generator.flow_from_directory(
    directory=input_path+'val',
    target_size=(image_dims, image_dims),
    batch_size=batch_size,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [9]:
# Train the model
epochs = 5
history = model.fit(
    training_gen,
    steps_per_epoch=10,
    epochs=epochs,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)  
)

Epoch 1/5
10/10 [==============================] - 37s 3s/step - loss: 0.5726 - accuracy: 0.7344 - val_loss: 0.7714 - val_accuracy: 0.5000
Epoch 2/5
10/10 [==============================] - 37s 4s/step - loss: 0.5544 - accuracy: 0.7375 - val_loss: 0.5629 - val_accuracy: 0.8750
Epoch 3/5
10/10 [==============================] - 37s 4s/step - loss: 0.5188 - accuracy: 0.7609 - val_loss: 0.4918 - val_accuracy: 0.7500
Epoch 4/5
10/10 [==============================] - 32s 3s/step - loss: 0.3617 - accuracy: 0.8536 - val_loss: 0.5408 - val_accuracy: 0.6875
Epoch 5/5
10/10 [==============================] - 35s 3s/step - loss: 0.2971 - accuracy: 0.8656 - val_loss: 0.4993 - val_accuracy: 0.7500


In [10]:
# Test data generator
test_data_generator = ImageDataGenerator(rescale=1./255)
test_gen = test_data_generator.flow_from_directory(
    directory=input_path+'test',
    target_size=(image_dims, image_dims),
    batch_size=128,
    class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [11]:
# Evaluate the model on test data
eval_result = model.evaluate(test_gen)
print('Loss on evaluation data:', eval_result[0])
print('Accuracy on evaluation data:', eval_result[1])

5/5 [==============================] - 13s 3s/step - loss: 0.4347 - accuracy: 0.7933
Loss on evaluation data: 0.43469586968421936
Accuracy on evaluation data: 0.7932692170143127


In [13]:
model.save('Pneumonia_Detection_model.keras')